## Instalando dependencias necesarias

In [1]:
%pip install -U sentence-transformers pandas


Note: you may need to restart the kernel to use updated packages.


## Entendiendo el dataset

In [12]:
# https://www.kaggle.com/datasets/omarhanyy/imdb-top-1000?resource=download
# TODO: Cargar el archivo del dataset con  pandas

import pandas as pd
from sentence_transformers import SentenceTransformer, util
df = pd.read_csv(r'D:\Documents\semantic_search\IMDB top 1000.csv')

In [14]:
# TODO: mostrar los primeros 5 registros de dataframe
df.head()

,Unnamed: 0,Title,Certificate,Duration,Genre,Rate,Metascore,Description,Cast,Info
0,0,1. The Shawshank Redemption (1994),R,142 min,Drama,9.3,80.0,Two imprisoned men bond over a number of years...,"Director: Frank Darabont | Stars: Tim Robbins,...","Votes: 2,295,987 | Gross: $28.34M"
1,1,2. The Godfather (1972),R,175 min,"Crime, Drama",9.2,100.0,The aging patriarch of an organized crime dyna...,Director: Francis Ford Coppola | Stars: Marlon...,"Votes: 1,584,782 | Gross: $134.97M"
2,2,3. The Dark Knight (2008),PG-13,152 min,"Action, Crime, Drama",9.0,84.0,When the menace known as the Joker wreaks havo...,Director: Christopher Nolan | Stars: Christian...,"Votes: 2,260,649 | Gross: $534.86M"
3,3,4. The Godfather: Part II (1974),R,202 min,"Crime, Drama",9.0,90.0,The early life and career of Vito Corleone in ...,Director: Francis Ford Coppola | Stars: Al Pac...,"Votes: 1,107,253 | Gross: $57.30M"
4,4,5. The Lord of the Rings: The Return of the Ki...,PG-13,201 min,"Action, Adventure, Drama",8.9,94.0,Gandalf and Aragorn lead the World of Men agai...,"Director: Peter Jackson | Stars: Elijah Wood, ...","Votes: 1,614,369 | Gross: $377.85M"


## Usando Sentence Transformer para crear embeddings

In [18]:
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

C:\Users\CN\anaconda3\Lib\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [20]:
embeddings = model.encode(df['Title'].tolist(), batch_size=64, show_progress_bar=True)

Batches:   0%|          | 0/16 [00:00<?, ?it/s]

In [22]:
df['embeddings'] = embeddings.tolist()

## Calculando la similitud usando la métrica de similitud por coseno

In [24]:
def compute_similarity(example, query_embedding):
    embedding = example['embeddings'] 
    similarity = util.cos_sim(embedding, query_embedding).item()
    return similarity  


## Ejecuntando la búsqueda 

In [26]:
query_embedding = model.encode(['a travel time adventure'])[0]
df['similarity'] = df.apply(lambda x: compute_similarity(x, query_embedding), axis=1)
df = df.sort_values(by='similarity', ascending=False)

df.head()['Title']

798    301. Nights of Cabiria (1957)
498    301. Nights of Cabiria (1957)
298    301. Nights of Cabiria (1957)
898    301. Nights of Cabiria (1957)
698    301. Nights of Cabiria (1957)
Name: Title, dtype: object